In [9]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 7, 4)

### Today must be last business day

In [10]:
today = date(2022, 7, 1)
today

datetime.date(2022, 7, 1)

In [4]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

### New dividend stock

In [11]:
name = 'MC'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [12]:
sqlIns = """
INSERT INTO dividend
VALUES('MC',0.32,0,0.20,0,0.52,0,0,00,'2022-02-24','2022-03-10','',1)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [17]:
def update_dividend1(q2,XDATE,PAIDDATE,actual,name):
    sql = "UPDATE dividend SET q2 = %s, dividend = q1+q2+q3+q4, XDATE = '%s', PAIDDATE = '%s', actual = %s WHERE name = '%s'"
    sql = sql % (q2,XDATE, PAIDDATE, actual,name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [20]:
name = 'SCCC'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,SCCC,9.0000,0.0000,9.0000,0.0000,18.0000,"1,200",2022-08-13,2022-08-28,,0


In [19]:
q2 = 9.00
XDATE = '2022-08-13'
PAIDDATE = '2022-08-28'
actual = 0

update_dividend1(q2,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [41]:
def update_dividend2(shares,q2,actual,name):
    sql = "UPDATE dividend SET shares = %s, q2 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q2, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [89]:
name = 'WHAIR'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHAIR,0.1738,0.1738,0.1738,0.1556,0.6770,0,2022-05-25,2022-06-22,,0


In [90]:
shares = 40000
q2 = 0.1738
actual = 0

In [91]:
update_dividend2(shares,q2,actual,name)

'Records updated = 1'

### Toggle actual status

In [18]:
name = 'SCB'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [10]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("MC")
'''
rp = const.execute(sqlDel)
rp.rowcount

1

### Start of output to cloud

In [120]:
sql = """
SELECT Y.NAME AS name,  Q2 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q2 > 0 
AND P.date = '%s'
ORDER BY XDATE
"""
sql = sql % today 
print(sql)


SELECT Y.NAME AS name,  Q2 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q2 > 0 
AND P.date = '2022-07-01'
ORDER BY XDATE



In [121]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [122]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].sort_values('xdate',ascending=True).style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,DIF,0.2610,"40,000","10,440.00","9,396.00",2022-08-10,2022-09-03,"588,000.00",1.60%,0
1,SCC,8.5000,900,"7,650.00","6,885.00",2022-08-13,2022-08-27,"358,200.00",1.92%,0
2,SCCC,9.0000,"1,200","10,800.00","9,720.00",2022-08-13,2022-08-28,"205,200.00",4.74%,0
3,WHART,0.1915,"30,000","5,745.00","5,170.50",2022-08-17,2022-09-02,"351,000.00",1.47%,0
4,WHAIR,0.1738,"40,000","6,952.00","6,256.80",2022-08-19,2022-09-22,"358,000.00",1.75%,0
5,JASIF,0.2400,"120,000","28,800.00","25,920.00",2022-08-20,2022-09-07,"1,209,000.00",2.14%,0
6,DCC,0.0500,"60,000","3,000.00","2,700.00",2022-08-23,2022-09-03,"177,600.00",1.52%,0
7,STA,1.2500,"22,500","28,125.00","25,312.50",2022-08-23,2022-09-08,"759,375.00",3.33%,0
8,KCE,0.6000,"15,000","9,000.00","8,100.00",2022-08-23,2022-09-08,"1,076,250.00",0.75%,0
9,NER,0.0700,"27,000","1,890.00","1,701.00",2022-08-23,2022-09-06,"201,150.00",0.85%,0


In [119]:
name = 'WHART'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.1195,0.2553,0.1915,0.1915,0.7578,"30,000",2022-08-17,2022-09-02,,0


In [118]:
q2 = 0.1915
XDATE = '2022-08-17'
PAIDDATE = '2022-09-02'
actual = 0

update_dividend1(q2,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [123]:
dividend.net.sum()

231018.3

In [124]:
file_name = 'dividend-q2.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_excel(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(box_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(one_file, index=False)

### End of output to cloud